In [1]:
# using Pkg
# Pkg.add("PyCall")
# ENV["PYTHON"] = "/home/alexmy/JULIA/SGS/SGS/venv/bin/python"
# Pkg.build("PyCall")
include("src/graph.jl")
# include("src/sets.jl")

Main.HllGraph

In [2]:
# using PyCall
# pandas  = pyimport("pandas")
# redis   = pyimport("redis")
# sqlite  = pyimport("sqlite3")
# csv     = pyimport("csv")

csv_file_path = "/home/alexmy/Downloads/POC/DATA/enron_05_17_2015_with_labels_v2.csv"

using CSV
using DataFrames
using WordTokenizers

using ..HllGraph
using ..HllSets

In [3]:
"""
    1. Message-ID	
    2. Date	
    3. From	
    4. To	
    5. Subject	
    6. X-From	X-To	X-cc	X-bcc	X-Folder	X-Origin	X-FileName	
    13. content	
    14. user	
    15. Cat_1_level_1	Cat_1_level_2	Cat_1_weight	Cat_2_level_1	Cat_2_level_2	Cat_2_weight	Cat_3_level_1	Cat_3_level_2	Cat_3_weight	Cat_4_level_1	Cat_4_level_2	Cat_4_weight	Cat_5_level_1	Cat_5_level_2	Cat_5_weight	Cat_6_level_1	Cat_6_level_2	Cat_6_weight	Cat_7_level_1	Cat_7_level_2	Cat_7_weight	Cat_8_level_1	Cat_8_level_2	Cat_8_weight	Cat_9_level_1	Cat_9_level_2	Cat_9_weight	Cat_10_level_1	Cat_10_level_2	Cat_10_weight	Cat_11_level_1	Cat_11_level_2	Cat_11_weight	Cat_12_level_1	Cat_12_level_2	Cat_12_weight	
    51. labeled
"""

df      = DataFrame(CSV.File(csv_file_path, header=true))
df.From = map(x -> ismissing(x) ? "" : (isnothing(match(r"'([^']*)'", x)) ? "" : match(r"'([^']*)'", x).captures[1]), df.From)
df.To   = map(x -> ismissing(x) ? "" : (isnothing(match(r"'([^']*)'", x)) ? "" : match(r"'([^']*)'", x).captures[1]), df.To)

df_filled = coalesce.(df, "unknown")

# Initialize sets to collect data from specific columns
hll_03 = HllSets.HllSet{10}()     # Set{String}()
hll_04 = HllSets.HllSet{10}()     # Set{String}()
hll_05 = HllSets.HllSet{10}()     # Set{String}()
hll_06 = HllSets.HllSet{10}()     # Set{String}()
hll_14 = HllSets.HllSet{10}()     # Set{String}()
hll_15 = HllSets.HllSet{10}()     # Set{String}()

HllSet{10}()


In [4]:
using Dates
using Redis
conn = Redis.RedisConnection()

# # Read the CSV file
i = 0
total = 0
println("procecessed: ", total, " rows")
start_time = Dates.now()

tokenized_csv_path = [String(s) for s in nltk_word_tokenize(csv_file_path)]
file = HllGraph.create_node(tokenized_csv_path, "node,file")
file_sha1 = file.sha1
HllGraph.set_node(conn, file, "b")

pipeline = open_pipeline(conn)

for row in eachrow(df_filled)
    hll_row = HllSets.HllSet{10}() 
    
    row_3 = nltk_word_tokenize(row[3])
    HllSets.add!(hll_03, row_3)    
    HllSets.add!(hll_row, row_3) 

    row_4 = nltk_word_tokenize(row[4])
    HllSets.add!(hll_04, row_4) 
    HllSets.add!(hll_row, row_4)

    row_5 = nltk_word_tokenize(row[5])
    HllSets.add!(hll_05, row_5)
    HllSets.add!(hll_row, row_5) 

    row_6 = nltk_word_tokenize(row[6])
    HllSets.add!(hll_06, row_6)
    HllSets.add!(hll_row, row_6) 

    row_14 = nltk_word_tokenize(row[14])
    HllSets.add!(hll_14, row_14) 
    HllSets.add!(hll_row, row_14) 

    row_15 = nltk_word_tokenize(row[15])
    HllSets.add!(hll_15, row_15)
    HllSets.add!(hll_row, row_15)

    row_node = HllGraph.Node(HllSets.id(hll_row), "node,row,$file_sha1", HllSets.dump(hll_row))
    key_name, node_dict = HllGraph.set_node(conn, row_node, "b", true)

    Redis.hmset(pipeline, key_name, node_dict)

    i += 1
    if i > 10000
        responses = read_pipeline(pipeline)
        pipeline = open_pipeline(conn)
        total += i
        end_time = Dates.now()
        println("processed: ", total, " rows in ", end_time - start_time, " seconds")
        start_time = Dates.now()
        i = 0
    end
end

responses = read_pipeline(pipeline)


procecessed: 0 rows
processed: 10001 rows in 158713 milliseconds seconds
processed: 20002 rows in 156921 milliseconds seconds


LoadError: InterruptException:

In [ ]:


node2 = HllGraph.Node(HllSets.id(hll_02), "node,column", HllSets.dump(hll_02))
HllGraph.set_node(conn, node2, "b")

node3 = HllGraph.Node(HllSets.id(hll_03), "node,column", HllSets.dump(hll_03))
HllGraph.set_node(conn, node3, "b")

node4 = HllGraph.Node(HllSets.id(hll_04), "node,column", HllSets.dump(hll_04))
HllGraph.set_node(conn, node4, "b")

node5 = HllGraph.Node(HllSets.id(hll_05), "node,column", HllSets.dump(hll_05))
HllGraph.set_node(conn, node5, "b")

node13 = HllGraph.Node(HllSets.id(hll_13), "node,column", HllSets.dump(hll_13))
HllGraph.set_node(conn, node13, "b")

node14 = HllGraph.Node(HllSets.id(hll_14), "node,column", HllSets.dump(hll_14))
HllGraph.set_node(conn, node14, "b")

In [ ]:
println(length(Set(set_02)))
println(Set(set_02))